## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/reviews-2.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

reviews,Name,Platform,metacritic_score,Num_of_comments
"Just as a Nintendo 64 without 'Super Mario 64' was unthinkable, so too is a Wii without Wii Sports. It’s one of the most perfect launch games we’ve seen.",Wii Sports,wii,90,49
"In terms of control and immersion, Wii Sports destroys the competition and provides an excellent foundation for other developers to build upon.",Wii Sports,wii,90,49
Wii Sports is the perfect pack-in for the Wii since it really shows off the system’s unique capabilities and manages to draw in casual gamers who would otherwise not want to play videogames.,Wii Sports,wii,87,49
The game does a great job of showing off what the system is about and showcasing the system’s abilities.,Wii Sports,wii,87,49
"We can't help but enjoy the fact that we're getting a solid sports experience for nothing. It's definitely more fun in groups and won't have a great deal of longevity, but anyone who doesn't find at least some fun in Wii Sports has a heart of coal.",Wii Sports,wii,85,49
"Sure, it lacks goals and can be beaten very quickly. But the ultra-responsive technology and high multiplayer replay value are far greater than any other party or sports game collection.",Wii Sports,wii,85,49
"The single-player game is good for practice and daily tests, but Wii Sports shines as a multiplayer game. [Jan. 2007, p.96]",Wii Sports,wii,83,49
"It's bloody good fun and you'll enjoy playing it for ages. [January 2007, p.38]",Wii Sports,wii,82,49
"It's a game you can play with your friends and family, and it perfectly highlights the direction that Nintendo is taking with the platform. It's not the best-looking game on the system, and it's definitely not the deepest, but it may well be the most fun.",Wii Sports,wii,80,49
"This is a simple game about simple fun, and everyone owes it to themselves to try it out to get a taste for what the Wii may in fact have in store for us all very soon.",Wii Sports,wii,80,49


In [0]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.sql.functions import col, udf, length
from pyspark.sql.types import IntegerType

count_tokens = udf(lambda words: len(words), IntegerType())

In [0]:
df = df.withColumn("length", length(col("reviews")))

In [0]:
tokenizer= Tokenizer(inputCol="reviews", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashing = HashingTF(inputCol="filtered", outputCol="hashed")
idf = IDF(inputCol="hashed", outputCol="tf_idf")


In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
clean_up = VectorAssembler(inputCols=["tf_idf","length"], outputCol="features")

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_prep_pipe = Pipeline(stages=[tokenizer, remover, hashing, idf, clean_up])

In [0]:
cleaner = data_prep_pipe.fit(df)

In [0]:
cleaned_data = cleaner.transform(df)

In [0]:
cleaned_data.columns

Out[12]: ['reviews',
 'Name',
 'Platform',
 'metacritic_score',
 'Num_of_comments',
 'length',
 'words',
 'filtered',
 'hashed',
 'tf_idf',
 'features']

In [0]:
clean_data = cleaned_data.select("metacritic_score", "features")

In [0]:
clean_data = clean_data.withColumnRenamed("metacritic_score","label")

In [0]:
training, test = clean_data.randomSplit([0.7,0.3])

In [0]:
rf = RandomForestRegressor(featuresCol="features")

In [0]:
model = rf.fit(training)

In [0]:
predictions= model.transform(test)

predictions.select("prediction","label","features").show(5)

+-----------------+-----+--------------------+
 prediction|label| features|
+-----------------+-----+--------------------+
73.40516263052953| 20|(262145,[2710,142...|
73.40516263052953| 20|(262145,[8721,330...|
73.40516263052953| 40|(262145,[2548,328...|
73.40516263052953| 40|(262145,[3467,825...|
75.87639091548459| 40|(262145,[3692,315...|
+-----------------+-----+--------------------+
only showing top 5 rows

In [0]:
evaluator = RegressionEvaluator(
  labelCol="label", predictionCol="prediction",metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Root Mean Squared Error (RMSE) on test data = 14.736

In [0]:
 from pyspark.ml.regression import LinearRegression

In [0]:
lr = LinearRegression(featuresCol="features", labelCol="label", maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(training)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: (262145,[369,1696,2437,2548,3048,3692,4214,4249,4319,4558,4900,5069,5381,5561,5798,6346,7002,7340,7625,9253,9588,9657,9886,9958,11382,11422,11941,12524,12843,14118,15336,15539,15585,15960,17291,17717,17949,18176,18337,18596,19153,19212,19517,19633,20263,21534,21823,24856,25456,26833,26944,27971,28183,28231,28760,28902,29514,29553,30006,30014,30172,30789,31536,31571,31766,31850,32550,32663,32890,33123,33532,33696,33816,34169,34519,35450,36038,36225,36702,36998,37371,37409,37542,38146,38655,41006,41333,41933,43899,44211,45190,45512,49980,50671,51004,51504,52154,52226,52351,52366,52437,52471,52774,53451,54222,54558,55655,55875,58348,59204,60080,61478,61595,62696,62829,63548,63617,63783,64096,64705,64748,65482,66379,66586,67058,67138,67202,67662,68115,68129,68228,68847,69318,69523,70456,71314,72431,72553,72837,73249,74271,75142,75879,77806,78329,78612,78745,78761,78833,79207,79584,81229,81386,81662,82322,82970,83350,83437,84738,84796,84871,85041,85120,85735,86399,86469,86478,86480,86807,88505,89833,90636,91182,92869,92925,93481,94617,94793,95122,95430,96266,96454,97096,97416,98142,98221,98416,100555,100872,101305,101336,101496,102742,103063,103218,103326,103367,103775,104472,104849,105093,105812,105938,105949,106484,106620,107508,107514,107810,107845,107951,108258,108278,109165,109753,110357,110585,110735,110837,111687,111767,111892,112493,112747,113004,113117,115082,115429,115934,116996,117303,118681,118787,118957,119530,119778,120069,120295,120380,121359,121601,123147,123541,123825,123940,124038,125502,125638,125747,126368,127702,127949,128073,129299,129583,129728,130365,130829,131400,133774,134579,134711,135642,136122,136616,136634,136951,137842,139418,139689,139916,140214,140322,142436,142931,144094,144764,145380,146511,146982,147136,147328,147452,148045,148921,149300,149982,150058,153946,153969,154224,154389,155236,155622,155719,157251,158661,159015,159775,160429,160622,160692,160893,161634,161931,162353,162383,162736,163240,163685,163802,164432,165164,165554,166027,166242,166735,167207,167301,168101,168139,168434,168590,169300,169312,169691,169944,170414,170535,171407,171917,172284,172621,173415,175801,176497,177231,177524,178139,180147,181708,181750,181799,182021,183261,183351,183997,184195,184349,184641,185276,185423,185450,186280,186480,186756,186925,188518,188835,189057,190632,191386,191476,192045,192390,192420,193059,193832,194296,194455,194802,195055,195108,196098,196226,196580,198389,199176,199247,199468,199496,199581,199667,199845,199937,200160,201386,201465,201738,202290,203260,203802,204158,204974,205234,205660,206173,206231,207438,207797,207889,208249,208892,209304,209825,210223,210689,210936,211154,211954,211971,212368,214068,214814,214982,215083,216109,216369,216372,216998,217140,217363,217492,218624,219394,219897,220129,220646,221285,221315,221883,222808,222930,224074,224500,226126,226953,227615,229407,230424,230980,230996,232205,232324,232735,232944,233148,233343,234706,235273,235351,235803,236263,237340,237720,239221,239279,239452,239740,239857,239947,242022,243200,243983,244114,244143,244435,245234,245429,245484,246559,246866,247740,247869,248002,248730,249550,249598,249848,250221,250434,250858,251012,251419,251726,251932,252103,252343,253066,253181,253522,254292,254359,255519,255526,255581,255868,256618,256970,257571,257873,258463,258646,258846,259091,259097,259145,259558,260659,261291,261870,262009,262099,262144],[-0.39786984723314933,-0.037041695425475335,-0.0033452239489577883,-0.021729983276069845,-0.16640366222386058,-0.11606514610164519,0.012824830165598006,-0.22253652213857825,0.8316266671441112,0.22844648109301616,-0.2764623735068385,0.028220968929935662,0.02846219247607895,0.23282593744416546,-0.009993823925950688,0.15387751516897752,0.13031159849877025,0.3778094394357726,0.14905704340149833,-0.5964394102382791,-0.0932841224347677,-1.0641391778059295,0.01838868104794685,-0.06922818878147018,0.1608276604276553,0.20462386709303435,-1.0185593788239549,0.5072427504527919,-0.37876180222028

In [0]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)


RMSE: 13.270314
r2: 0.250413

In [0]:
lr_predictions = lr_model.transform(test)
lr_predictions.select("prediction", "label", "features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction",\
                                  labelCol="label", metricName="r2")


+-----------------+-----+--------------------+
 prediction|label| features|
+-----------------+-----+--------------------+
73.54007690760515| 20|(262145,[33075,33...|
78.13164560648005| 20|(262145,[33075,50...|
74.07559581312262| 30|(262145,[1226,105...|
74.82874369979149| 30|(262145,[2437,700...|
67.43890333341204| 30|(262145,[8956,464...|
+-----------------+-----+--------------------+
only showing top 5 rows

In [0]:
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

test_result = lr_model.evaluate(test)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)



R Squared (R2) on test data = 0.243572
Root Mean Squared Error (RMSE) on test data = 13.3571

In [0]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()

numIterations: 11
objectiveHistory: [0.4999963228534578, 0.4839952211386509, 0.44412447470909816, 0.43771124670852646, 0.4367456052690371, 0.4362590845469373, 0.4361453878679764, 0.43613338467516644, 0.43612788412244935, 0.4361270038941448, 0.436126738181945]
+-------------------+
 residuals|
+-------------------+
 -57.16539892159422|
-47.518464836775905|
 -54.53733632908212|
 -36.47395550064702|
-43.658590070947994|
 -39.20848791543466|
 -44.00308941907279|
 -40.15343428737475|
 -35.67777274660111|
-26.726132455626143|
-33.443848862753995|
-31.425772489284014|
-30.473673274443968|
-23.442573322141335|
 -33.47958769939838|
 -35.78830783236869|
 -32.08352307555619|
 -39.14555740318153|
 -33.79259381215225|
 -35.10923223655671|
+-------------------+
only showing top 20 rows